# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe:
    - Kacem Khaled
    - Oumayma Messoussi
    - Semah Aissaoui




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [127]:
import numpy as np
import math
import copy
from collections import deque

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        # TODO
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        s = copy.deepcopy(self)
        s.prev = self
        s.rock = rock_pos
        return s
            
    def score_state(self,rh):
        self.score = 10 * self.pos[0] - self.estimee55(rh)
        if self.c == self.prev.c and self.d == -self.prev.d:
            self.score -= 50
        return self.score

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)
    
    """
    estimee1: Estimation du nombre de coups restants 
    """
    def estimee1(self):
        return 4 - self.pos[0]
        
        
    def move_blocker_away_cost(self,rh,blocked,blocker):
        move_away_cost = 0
        move_away_cost_up = move_away_cost_down = 0
        if rh.move_on[blocked] - rh.length[blocker] >= 0:        #can we move it up / left?
            move_away_cost_up = self.pos[blocker]+rh.length[blocker]-rh.move_on[blocked]
        if rh.move_on[blocked] + rh.length[blocker] <= 5:        #can we move it down /right?
            move_away_cost_down = 1 - self.pos[blocker]+rh.move_on[blocked]
        if move_away_cost_up == 0 and move_away_cost_down > 0:
            move_away_cost += move_away_cost_down
        elif move_away_cost_up > 0 and move_away_cost_down == 0:
            move_away_cost += move_away_cost_up
        else:
            move_away_cost += min(move_away_cost_up,move_away_cost_down)
        return move_away_cost
    
    def estimee5(self, rh):
        L1,S1 = rh.blocking_cars(self)
        L2,S2 = rh.blocking_cars_level_2(self)
        L3,S3 = rh.blocking_cars_level_3(self)
        return self.estimee1()+S1+S2+S3+ 5*len(L1) +4*len(L2) +3*len(L3) +2*len(rh.blocking_cars_level_4(self)) + 1*len(rh.blocking_cars_level_5(self))
        
    def estimee55(self, rh):
        
        L2,S2 = rh.blocking_cars_level_2(self)
        L3,S3= rh.blocking_cars_level_3(self)
        length = 0
        list_blockers, move_away_score,blockers_counting_score = rh.blocking_cars_level_general(self,5)
        return self.estimee1()+move_away_score + blockers_counting_score
    
    


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [110]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock != None:
            self.free_pos[state.rock[0]][state.rock[1]] = False
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        #TODO
        for i in range(6):
            if i == 2: continue
            for j in range(6):
                if self.free_pos[i][j]:
                    if state.rock == None:
                        new_states.append(state.put_rock((i,j)))
                    elif state.rock[0] != i and state.rock[1] != j:
                        new_states.append(state.put_rock((i,j)))
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock != None:
            grid[state.rock] = 'x'
        print(grid)
        
    
    
    
    #def car_blockers(self,state,concerned)
        
    def blocking_cars(self,state):
        block_list = set()
        score = 0
        for i in range(self.nbcars):
            if not self.horiz[i]:
                if (state.pos[i]+self.length[i]-1 >= self.move_on[0]) and (state.pos[i] <= self.move_on[0]):
                    if self.move_on[i] > state.pos[0]+self.length[0]-1:
                        block_list.add(i)
                        score += state.move_blocker_away_cost(rh,0,i)
        return block_list,score
    
    def blocking_cars_level_2(self,state):
        block_list,_ = self.blocking_cars(state)
        block_list_level_2 = set()
        score = 0
        for i in range(1,self.nbcars):
            #if i in block_list: continue
            for blocker in block_list:
                if self.horiz[i]:
                    if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker]) and (state.pos[i] <= self.move_on[blocker]):
                        if ((self.move_on[i] > state.pos[blocker]+self.length[blocker]-1 ) and \
                            (self.move_on[i] <= self.move_on[0]+self.length[blocker])) or \
                        ((self.move_on[i] < state.pos[blocker]) and \
                         (self.move_on[i] >= self.move_on[0] - self.length[blocker])):
                            block_list_level_2.add(i)
                            score += state.move_blocker_away_cost(rh,blocker,i)
        return block_list_level_2,score
    
    def blocking_cars_level_general(self,state,depth):
        blockers = []
        length_score = 0
        #score = 0
        block_list,score1 = self.blocking_cars(state)
        block_list_level_2,score2 = self.blocking_cars_level_2(state)
        score = score1 + score2
        vertical = True
        if block_list: 
            blockers.append(block_list)
            length_score+= depth * len(block_list)
        if block_list_level_2: 
            blockers.append(block_list_level_2)
            length_score+= (depth-1) * len(block_list_level_2)
        count = 2
        while block_list_level_2:
            #depth -= count
            block_list_level_3 = set()
            for i in range(1,self.nbcars):
                #if i in block_list: continue
                for blocker in block_list:
                    #if blocker in block_list_level_2: continue
                    for blocker_level_2 in block_list_level_2:
                        if ( (not self.horiz[i] and vertical) or ( self.horiz[i] and  not vertical)) :
                            if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_2]) and (state.pos[i] <= self.move_on[blocker_level_2]):
                                if ((self.move_on[i] > state.pos[blocker_level_2]+self.length[blocker_level_2]-1 ) and \
                                    (self.move_on[i] <= self.move_on[blocker]+self.length[blocker_level_2])) or \
                                ((self.move_on[i] < state.pos[blocker_level_2]) and \
                                 (self.move_on[i] >= self.move_on[blocker] - self.length[blocker_level_2])):
                                    block_list_level_3.add(i)
                                    if (depth - count) >= 3: score +=  state.move_blocker_away_cost(rh,blocker,i)
            
            block_list = block_list_level_2
            block_list_level_2 = block_list_level_3
            vertical = not vertical
            if block_list_level_3 and len(blockers)< depth:
                blockers.append(block_list_level_3)
                length_score+= (depth-count) * len(block_list_level_3)
                #print("appended new level "+str(count)+" = "+str((depth-count) * len(block_list_level_3)))
                count+=1
            else: 
                #print("break from general")
                break
        return blockers,score,length_score
    
    """ level 3 , level 4 , level 5 of blocking cars written explicitely for verification purposes"""    

    def blocking_cars_level_3(self,state):
        score = 0
        block_list,_ = self.blocking_cars(state)
        block_list_level_2 = set()
        block_list_level_2,a = self.blocking_cars_level_2(state)
        block_list_level_3 = set()
        for i in range(self.nbcars):
            for blocker in block_list:
                for blocker_level_2 in block_list_level_2:
                    if not self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_2]) and (state.pos[i] <= self.move_on[blocker_level_2]):
                            if ((self.move_on[i] > state.pos[blocker_level_2]+self.length[blocker_level_2]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker]+self.length[blocker_level_2])) or \
                            ((self.move_on[i] < state.pos[blocker_level_2]) and \
                             (self.move_on[i] >= self.move_on[blocker] - self.length[blocker_level_2])):
                                block_list_level_3.add(i)
                                score += state.move_blocker_away_cost(rh,blocker,i)
        return block_list_level_3,score
    
    def blocking_cars_level_4(self,state):
        block_list_level_2, _ = self.blocking_cars_level_2(state)
        block_list_level_3, _ = self.blocking_cars_level_3(state)
        block_list_level_4 = set()
        for i in range(1,self.nbcars):
            for blocker_level_2 in block_list_level_2:
                for blocker_level_3 in block_list_level_3:
                    if self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_3]) and (state.pos[i] <= self.move_on[blocker_level_3]):
                            if ((self.move_on[i] > state.pos[blocker_level_3]+self.length[blocker_level_3]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker_level_2]+self.length[blocker_level_3])) or \
                            ((self.move_on[i] < state.pos[blocker_level_3]) and \
                             (self.move_on[i] >= self.move_on[blocker_level_2] - self.length[blocker_level_3])):
                                block_list_level_4.add(i)
        return block_list_level_4
    
    def blocking_cars_level_5(self,state):
        block_list_level_3,_ = self.blocking_cars_level_3(state)
        block_list_level_4 = self.blocking_cars_level_4(state)
        block_list_level_5 = set()
        for i in range(1,self.nbcars):
            for blocker_level_3 in block_list_level_3:
                for blocker_level_4 in block_list_level_4:
                    if not self.horiz[i]:
                        if (state.pos[i]+self.length[i]-1 >= self.move_on[blocker_level_4]) and (state.pos[i] <= self.move_on[blocker_level_4]):
                            if ((self.move_on[i] > state.pos[blocker_level_4]+self.length[blocker_level_4]-1 ) and \
                                (self.move_on[i] <= self.move_on[blocker_level_3]+self.length[blocker_level_4])) or \
                            ((self.move_on[i] < state.pos[blocker_level_4]) and \
                             (self.move_on[i] >= self.move_on[blocker_level_3] - self.length[blocker_level_4])):
                                block_list_level_5.add(i)
        return block_list_level_5



### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [3]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [4]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [128]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth

    def minimax_1(self, current_depth, current_state): #rh 
        #TODO
        #if current_state.success():
        #    print("found success")

        if current_state.success() or current_depth == 0 :
            #print("reached limit in depth")
            return current_state,current_state.score_state(self.rushhour)
        best_move = -10000 
        children=self.rushhour.possible_moves(current_state)
        #random.shuffle(children)
        for child in children:
            s,score = self.minimax_1(current_depth - 1, child)
            if score > best_move:
                best_move = score
                best_state = s
        return best_state,best_move
        
    def decide_best_move_1(self):
        #TODO
        best_state,best_move = self.minimax_1(self.search_depth, self.state)
        self.state = self.state.move(best_state.c,best_state.d)
        #return best_state
            
    def minimax_2(self, current_depth, current_state, is_max): 
        #TODO
        return best_move

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        #TODO
        return best_move

    def expectimax(self, current_depth, current_state, is_max):
        #TODO
        return best_move

    
        

    def decide_best_move_2(self, is_max):
        #TODO
        return None

    def decide_best_move_pruning(self, is_max):
        # TODO
        return None

    def decide_best_move_expectimax(self, is_max):
        # TODO
        return None

    def solve(self, is_singleplayer):
        #TODO
        """
        #visited = set()
        fifo = deque([state])
        #visited.add(state)
        # TODO
        count =0
        while fifo:
            p=fifo.popleft()
            if p.success():
                #visited.add(p)
                print("Nombre de mouvements: "+str(count))
                return p
            chosen_child = self.decide_best_move_1(p)
            self.print_move(True,chosen_child)
            self.rushhour.print_pretty_grid(chosen_child)
            count+=1
            fifo.append(chosen_child)
        return None
    """
        if is_singleplayer:
            while not self.state.success():
                self.decide_best_move_1()
                self.print_move(True, self.state)
                self.rushhour.print_pretty_grid(self.state)
                self.solve(True)

        else:
            turn = True
            while not self.state.success():
                self.decide_best_move_1()
                self.print_move(turn, self.state)
                self.solve(turn)
                turn = not turn
            

    def print_move(self, is_max, state):
        #TODO
        fancy_prints = False
        if state:
            if is_max:
                if fancy_prints:
                    print("\n***new move***")
                    print("total score : "+str(state.score_state(self.rushhour)))
                    print("total estimee5 : "+str(state.estimee5(self.rushhour)))
                    print("score distance 'estimee1' : "+str(state.estimee1()))
                    listblock1,score1 = self.rushhour.blocking_cars(state)
                    print("1st blockers : 5 * len("+str(listblock1)+"); score1 = "+str(score1))
                    listblock2,score2 = self.rushhour.blocking_cars_level_2(state)
                    print("2nd blockers : 4 * len("+str(listblock2)+"); score2 = "+str(score2))
                    listblock3,score3 = self.rushhour.blocking_cars_level_3(state)
                    print("2nd blockers : 3 * len("+str(listblock3)+"); score3 = "+str(score3))
                    print("4th blockers : 2 * len("+str(self.rushhour.blocking_cars_level_4(state))+")")
                    print("5th blockers : 1 * len("+str(self.rushhour.blocking_cars_level_5(state))+")")
                    listblockG,scoreG,scoreL = self.rushhour.blocking_cars_level_general(state,5)
                    print("test blockers general : "+str(listblockG)+"; scoreG = "+str(scoreG)+"; scoreL = "+str(scoreL)+"; total = "+str(scoreL+scoreG) )               
                msg = str(self.state.nb_moves)+ ". Voiture "
                msg += self.rushhour.color[state.c] + " vers "
                if self.rushhour.horiz[state.c]:
                    if state.d == 1:
                        msg += "la droite"
                    else:
                        msg += "la gauche"
                else:
                    if state.d ==1:
                        msg += "le bas"
                    else:
                        msg += "le haut"
                print(msg)
            elif state.rock:
                msg = "Roche dans la case "+str(state.rock[0])+"-"+str(state.rock[1])
                print(msg)
        return None
          


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [23]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()


***new move***
total score : 7
total estimee5 : 3
score distance 'estimee1' : 3
1st blockers : 5 * len(set()); score1 = 0
2nd blockers : 4 * len(set()); score2 = 0
2nd blockers : 3 * len(set()); score3 = 0
4th blockers : 2 * len(set())
5th blockers : 1 * len(set())
test blockers general : []; scoreG = 0; scoreL = 0
1. Voiture rouge vers la droite

***new move***
Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [124]:
# Solution optimale: 9 moves
import random
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3

***new move***
appended new level 2 = 3
total score : -6
total estimee5 : 26
score distance 'estimee1' : 2
1st blockers : 5 * len({1, 2}); score1 = 4
2nd blockers : 4 * len({4}); score2 = 1
2nd blockers : 3 * len({3}); score3 = 2
4th blockers : 2 * len(set())
5th blockers : 1 * len(set())
appended new level 2 = 3
test blockers general : [{1, 2}, {4}, {3}]; scoreG = 7; scoreL = 17; total = 24
1. Voiture rouge vers la droite
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' 

In [122]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 3
appended new level 2 = 6
appended new level 3 = 2
appended new level 4 = 1
appended new level 2 = 3

***new move***
appended new level 2 = 3
total score : -24
total estimee5 : 34
score distance 'estimee1' : 3
1st blockers : 5 * len({6, 7}); score1 = 5
2nd blockers : 4 * len({4, 5}); score2 = 2
2nd blockers : 3 * len({3}); score3 = 3
4th blockers : 2 * len(set())
5th blockers : 1 * len(set())
appended new level 2 = 3
test blockers general : [{6, 7}, {4, 5}, {3}]; scoreG = 10; scoreL = 21; total = 31
1. Voiture emeraude vers la gauche
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [

In [129]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
appended new level 2 = 6
appended new level 3 = 4
appended new level 4 = 3
appended new level 2 = 6
appended new level 3 = 6
appended new level 4 = 4
appended new level 2 = 6
appended new level 3 = 4
appended new level 4 = 4
appended new level 2 = 6
appended new level 3 = 4
appended new level 4 = 2
appended new level 2 = 12
appended new level 3 = 10
appended new level 4 = 4
appended new level 2 = 6
appended new level 3 = 4
appended new level 4 = 3
appended new level 2 = 6
appended new level 3 = 4
appended new level 4 = 3
1. Voiture rouge vers la droite
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'-' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
appende

### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [57]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]

***new move***
total score : -12
total estimee5 : 22
score distance 'estimee1' : 3
1st blockers : 5 * len({1, 2}); score1 = 4
2nd blockers : 4 * len({4}); score2 = 1
2nd blockers : 3 * len(set()); score3 = 0
4th blockers : 2 * len(set())
5th blockers : 1 * len(set())
test blockers general : [{1, 2}, {4}]; scoreG = 5; scoreL = 14
1. Voiture orange vers le haut

***new move***
total score : -7
total estimee5 : 17
score distance 'estimee1' : 3
1st blockers : 5 * len({1, 2}); score1 = 4
2nd blockers : 4 * len(set()); score2 = 0
2nd blockers : 3 * len(set()); score3 = 0
4th blockers : 2 * len(set())
5th blockers : 1 * len(set())
test blockers general : [{1, 2}]; scoreG = 4; scoreL = 10
2. Voiture jaune vers la gauche
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'

In [10]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]

***new move***
total score : -19
score distance :3
score move away cost : 5
1st blockers : {6, 7}
2nd blockers : {4, 5}; score = 3
2nd blockers : set(); score = 0
4th blockers : set()
5th blockers : set()
test blockers general : [{6, 7}, {4, 5}]; scoreG = 3; scoreL = 18
1. Voiture orange vers le haut

***new move***
total score : -18
score distance :3
score move away cost : 4
1st blockers : {6, 7}
2nd blockers : {4, 5}; score = 3
2nd blockers : set(); score = 0
4th blockers : set()
5th blockers : set()
test blockers general : [{6, 7}, {4, 5}]; scoreG = 3; scoreL = 18
2. Voiture jaune vers le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'

In [158]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
***new move***
total score : 0
score distance :4
score move away cost : 6
1st blockers : {3, 4, 5}
2nd blockers : {2, 10}; score = 9
2nd blockers : {8, 1}; score = 3
4th blockers : {6, 7}
5th blockers : {8, 3, 4, 5}
test blockers general : [{3, 4, 5}, {2, 10}, {8, 1}, {0, 6, 7}, {8, 3, 4, 5}, {9, 2, 10, 6}, {1, 3, 4, 5, 8}]; scoreG = -1
1. Voiture 3 vers le bas
***new move***
total score : 0
score distance :3
score move away cost : 6
1st blockers : {3, 4, 5}
2nd blockers : {2, 10}; score = 9
2nd blockers : {8, 1}; score = 3
4th blockers : {6, 7}
5th blockers : {8, 3, 4, 5}
test blockers general : [{3, 4, 5}, {2, 10}, {8, 1}, {0, 6, 7}, {8, 3, 4, 5}, {0, 2, 6, 9, 10}, {1, 3, 4, 5, 8}]; scoreG = 24
2. Voiture rouge vers la droite
***new move***
total score 

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [194]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]

***new move***
total score : -9
score distance :2
score move away cost : 5
1st blockers : {1, 2}
2nd blockers : {4}; score = 1
2nd blockers : {3}; score = 2
4th blockers : set()
5th blockers : set()
test blockers general : [{1, 2}, {4}, {3}]; scoreG = 3; scoreL = 29
1. Voiture rouge vers la droite

***new move***
total score : -1
score distance :2
score move away cost : 3
1st blockers : {1}
2nd blockers : {4}; score = 1
2nd blockers : {3}; score = 2
4th blockers : set()
5th blockers : set()
test blockers general : [{1}, {4}, {3}]; scoreG = 3; scoreL = 21
2. Voiture bleu vers le haut

***new move***
total score : 5
score distance :2
score move away cost : 3
1st blockers : {1}
2nd blockers : {4}; score = 1
2nd blockers : set(); score = 0
4th blockers : set

In [270]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
***new move***
total score : 12
score distance :3
score move away cost : 4
1st blockers : {6, 7}
2nd blockers : {4, 5}
3rd blockers : {3}
4th blockers : set()
5th blockers : set()
test blockers : {6, 7}
test blockers general : [{6, 7}, {4, 5}, {3}]
Voiture bleu vers le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
***new move***
total score : 10
score distance :3
score move away cost : 2
1st blockers : {7}
2nd blockers : {4, 5}
3rd blockers : {3, 6}
4th blockers : set()
5th blockers : set()
test blockers : {7}
test blockers general : [{7}, {4, 5}, {3, 6}]
Voiture jaune vers le bas
[[b'v' b'v' b'-' b'-

In [271]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve( False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
***new move***
total score : 98
score distance :3
score move away cost : 6
1st blockers : {3, 4, 5}
2nd blockers : {2, 10}
3rd blockers : {8}
4th blockers : {6}
5th blockers : {5}
test blockers : {3, 4, 5}
test blockers general : [{3, 4, 5}, {2, 10}, {8}, {0, 6}, {1, 3, 4, 5}, {2, 6, 7, 9, 10}, {8, 5}, {0, 2, 10, 6}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}]
Voiture 1 vers le bas
[[b'-' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'1' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-'

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [272]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
***new move***
total score : 7
score distance :2
score move away cost : 2
1st blockers : {1}
2nd blockers : {4}
3rd blockers : {3}
4th blockers : set()
5th blockers : set()
test blockers : {1}
test blockers general : [{1}, {4}, {3}]
Voiture rouge vers la droite
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
***new move***
total score : 4
score distance :2
score move away cost : 1
1st blockers : {1}
2nd blockers : set()
3rd blockers : set()
4th blockers : set()
5th blockers : set()
test blockers : {1}
test blockers general : [{1}]
Voiture jaune vers la gauche
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-

In [273]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
***new move***
total score : 12
score distance :3
score move away cost : 4
1st blockers : {6, 7}
2nd blockers : {4, 5}
3rd blockers : {3}
4th blockers : set()
5th blockers : set()
test blockers : {6, 7}
test blockers general : [{6, 7}, {4, 5}, {3}]
Voiture lime vers la gauche
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'l' b'l' b'-']
 [b'o' b'-' b'-' b'e' b'e' b'e']]
***new move***
total score : 10
score distance :3
score move away cost : 2
1st blockers : {7}
2nd blockers : {4, 5}
3rd blockers : {3, 6}
4th blockers : set()
5th blockers : set()
test blockers : {7}
test blockers general : [{7}, {4, 5}, {3, 6}]
Voiture jaune vers le bas
[[b'v' b'v' b'-' 

In [274]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
***new move***
total score : 98
score distance :3
score move away cost : 6
1st blockers : {3, 4, 5}
2nd blockers : {2, 10}
3rd blockers : {8}
4th blockers : {6}
5th blockers : {5}
test blockers : {3, 4, 5}
test blockers general : [{3, 4, 5}, {2, 10}, {8}, {0, 6}, {1, 3, 4, 5}, {2, 6, 7, 9, 10}, {8, 5}, {0, 2, 10, 6}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}, {1, 3, 4, 5, 8}, {0, 2, 6, 7, 9, 10}]
Voiture 1 vers le bas
[[b'-' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'4' b'5']
 [b'1' b'r' b'r' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'-' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-'

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 